## Tensorflow使用feature_column银行营销分类数据集

### 演示流程：

1. 用Pandas导入 CSV 文件。
2. 用tf.dataset读取数据，进行分批（batch）、随机排序（shuffle）处理
3. 用feature_column将 CSV 中的列映射到用于训练模型的特征
4. 用 Keras 构建，训练并评估模型

### 银行营销数据集

数据来自葡萄牙银行机构电话营销活动的记录。

每行是同一个客户的电话沟通记录，以及最后一列是这客户最后明确购买还是不购买产品，是一个二分类问题。

数据列如下：
01. 年龄，数字
02. 工作类型：分类，比如管理员、企业家
03. 婚姻状况：分类，比如已婚、离婚、单身
04. 教育：分类，小学、中学等
05. 是否有信用：分类、是/否
06. 余额：数字、年均余额
07. 是否有住房贷款：分类、是/否
08. 是否有贷款：分类，是/否
09. 联系方式：分类、位置、手机号码、固定号码
10. 天：数字、最后一次联系日
11. 月：数字、最后一次联系的月份
12. 持续时间：数字、上次联系时间
13. 广告系列：数字，在这个广告和用户联系的次数，包括其他人
14. pdays：数字，和这个客户上次联系的间隔天数
15. 以前：数字，这个广告此前和此客户的联系次数
16. 以前的结果：分类、成功、失败
17. 最终是否订阅了定期存款：二分类、是、否

### 1. 导入库

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow'

### 2. 使用 Pandas读取CSV

In [ ]:
df = pd.read_csv("./datas/bank/bank-full.csv")
df.head()

### 3. 将 dataframe 拆分为训练、验证和测试集

In [4]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), '训练集数目')
print(len(val), '验证集数目')
print(len(test), '测试集数目')

193 训练集数目
49 验证集数目
61 测试集数目


### 4. 用tf.data.Dataset读取数据

In [5]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    """便捷函数，将pandas的df转换成dataset"""
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [6]:
batch_size = 32
train_ds = df_to_dataset(train, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [7]:
# 测试下数据集
for feature_batch, label_batch in train_ds.take(1):
    print(feature_batch)
    print()
    print(feature_batch["age"])
    print()
    print(label_batch)

{'age': <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([65, 45, 46, 59, 71, 67, 67, 48, 65, 47, 39, 55, 65, 57, 67, 29, 49,
       46, 62, 44, 56, 62, 54, 64, 59, 44, 45, 57, 50, 45, 56, 46])>, 'sex': <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 0])>, 'cp': <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 4, 2, 1, 2, 3, 4, 2, 4, 3, 3, 4, 4, 4, 4, 2, 4, 4, 4, 4, 2, 3,
       4, 4, 0, 2, 2, 2, 4, 4, 4, 4])>, 'trestbps': <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([160, 138, 105, 178, 160, 115, 125, 110, 110, 108,  94, 180, 135,
       110, 160, 130, 130, 140, 124, 120, 140, 130, 122, 120, 164, 130,
       130, 124, 150, 142, 200, 138])>, 'chol': <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([360, 236, 204, 270, 302, 564, 254, 229, 248, 243, 199, 327, 254,
       201, 286, 204, 269, 311, 209, 169, 294, 263, 286, 246, 176, 219,
       234, 261, 243, 309, 288, 243

### 5. feature_column特征处理


In [8]:
feature_columns = []

# 数值列
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# 分桶列
age = feature_column.numeric_column("age")
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# 分类列
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# 嵌入列
thal_embedding = feature_column.embedding_column(thal, dimension=4)
feature_columns.append(thal_embedding)

# 组合列
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=100)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

### 6. 创建，编译和训练模型

In [9]:
model = tf.keras.Sequential([
    layers.DenseFeatures(feature_columns),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=30)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

7/7 [==============================] - 0s 42ms/step - loss: 6.4846 - accuracy: 0.7150 - val_loss: 2.8021 - val_accuracy: 0.2857
Epoch 2/30
7/7 [==============================] - 0s 3ms/step - loss: 1.5845 - accuracy: 0.5596 - val_loss: 2.5465 - val_accuracy: 0.7143
Epoch 3/30
7/7 [==============================] - 0s 3ms/step - loss: 2.0645 - accuracy: 0.7150 - val_loss: 0.7416 - val_accuracy: 0.6327
Epoch 4/30
7/7 [==============================] - 0s 3ms/step - loss: 0.6792 - accuracy: 0.71

In [10]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 1ms/step - loss: 0.6322 - accuracy: 0.7377
Accuracy 0.7377049326896667
